# NY Taxi Dataset and Data Ingestion
https://github.com/DataTalksClub/data-engineering-zoomcamp/blob/main/01-docker-terraform/docker-sql/04-data-ingestion.md

In [1]:
import pandas as pd
# import pyarrow.parquet as pq

pd.__file__

'/home/ludo/Public/Projets_IA/my-data-engineering-zoomcamp/.venv/lib/python3.13/site-packages/pandas/__init__.py'

## Get the data

In [2]:
df = (
    pd.read_parquet(
        "green_tripdata_2025-11.parquet",
    )
    .drop(columns='ehail_fee')  # full of NaN values
    .astype({
        'RatecodeID': "Int64",
        'passenger_count': "Int64",
        'payment_type': "Int64",
        'trip_type': "Int64",
        'VendorID': "Int64",
        'PULocationID': "Int64",
        'DOLocationID': "Int64",
        # cbd_congestion_fee → new
    })
)
display(df.info())

<class 'pandas.DataFrame'>
RangeIndex: 46912 entries, 0 to 46911
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   VendorID               46912 non-null  Int64         
 1   lpep_pickup_datetime   46912 non-null  datetime64[us]
 2   lpep_dropoff_datetime  46912 non-null  datetime64[us]
 3   store_and_fwd_flag     41343 non-null  str           
 4   RatecodeID             41343 non-null  Int64         
 5   PULocationID           46912 non-null  Int64         
 6   DOLocationID           46912 non-null  Int64         
 7   passenger_count        41343 non-null  Int64         
 8   trip_distance          46912 non-null  float64       
 9   fare_amount            46912 non-null  float64       
 10  extra                  46912 non-null  float64       
 11  mta_tax                46912 non-null  float64       
 12  tip_amount             46912 non-null  float64       
 13  tolls_amount

None

In [3]:
display(df.head(10))

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-11-01 00:34:48,2025-11-01 00:41:39,N,1,74,42,1,0.74,7.2,1.00,0.5,1.94,0.0,1.0,11.64,1,1,0.00,0.00
1,2,2025-11-01 00:18:52,2025-11-01 00:24:27,N,1,74,42,2,0.95,7.2,1.00,0.5,0.00,0.0,1.0,9.70,2,1,0.00,0.00
2,2,2025-11-01 01:03:14,2025-11-01 01:15:24,N,1,83,160,1,2.19,13.5,1.00,0.5,5.00,0.0,1.0,21.00,1,1,0.00,0.00
3,2,2025-11-01 00:10:57,2025-11-01 00:24:53,N,1,166,127,1,5.44,24.7,1.00,0.5,0.50,0.0,1.0,27.70,1,1,0.00,0.00
4,1,2025-11-01 00:03:48,2025-11-01 00:19:38,N,1,166,262,1,3.20,18.4,3.75,1.5,1.00,0.0,1.0,24.65,1,1,2.75,0.00
5,1,2025-11-01 00:42:13,2025-11-01 01:04:50,N,1,112,48,2,5.10,26.8,4.50,1.5,6.55,0.0,1.0,39.35,1,1,2.75,0.75
6,2,2025-11-01 00:05:41,2025-11-01 00:39:20,N,1,83,87,1,9.80,43.6,1.00,0.5,9.92,0.0,1.0,59.52,1,1,2.75,0.75
7,2,2025-11-01 00:42:14,2025-11-01 01:13:20,N,1,66,233,1,5.01,28.9,1.00,0.5,6.98,0.0,1.0,41.88,1,1,2.75,0.75
8,2,2025-11-01 00:03:08,2025-11-01 00:06:27,N,1,223,223,1,0.63,5.1,1.00,0.5,1.52,0.0,1.0,9.12,1,1,0.00,0.00
9,2,2025-11-01 00:56:33,2025-11-01 01:01:34,N,1,130,130,1,1.15,7.9,1.00,0.5,0.00,0.0,1.0,10.40,2,1,0.00,0.00


In [4]:
display(df.tail(10))

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
46902,2,2025-11-30 17:45:00,2025-11-30 18:59:00,NaN,<NA>,42,132,<NA>,18.90,97.05,0.0,0.5,18.04,0.00,1.0,116.59,<NA>,<NA>,NaN,0.00
46903,6,2025-11-30 17:22:37,2025-11-30 18:09:41,NaN,<NA>,92,132,<NA>,9.46,0.00,0.0,0.5,0.00,0.00,0.3,36.85,<NA>,<NA>,NaN,0.00
46904,2,2025-11-30 18:01:00,2025-11-30 18:16:00,NaN,<NA>,65,181,<NA>,1.60,14.28,0.0,0.5,3.16,0.00,1.0,18.94,<NA>,<NA>,NaN,0.00
46905,6,2025-11-30 18:01:25,2025-11-30 18:37:16,NaN,<NA>,75,198,<NA>,8.80,5.80,0.0,0.5,0.00,6.94,0.3,37.71,<NA>,<NA>,NaN,0.00
46906,2,2025-11-30 18:31:00,2025-11-30 18:52:00,NaN,<NA>,74,142,<NA>,3.84,27.23,0.0,0.5,4.41,0.00,1.0,35.89,<NA>,<NA>,NaN,0.00
46907,2,2025-11-30 19:58:34,2025-11-30 20:14:28,NaN,<NA>,59,51,<NA>,8.50,33.22,0.0,0.5,0.00,0.00,1.0,34.72,<NA>,<NA>,NaN,0.00
46908,2,2025-11-30 19:34:00,2025-11-30 19:46:00,NaN,<NA>,74,151,<NA>,1.73,13.86,0.0,0.5,0.77,0.00,1.0,16.13,<NA>,<NA>,NaN,0.00
46909,2,2025-11-30 21:46:46,2025-11-30 22:17:55,NaN,<NA>,33,163,<NA>,7.52,38.42,0.0,0.5,1.00,0.00,1.0,44.42,<NA>,<NA>,NaN,0.75
46910,2,2025-11-30 21:00:00,2025-11-30 21:15:00,NaN,<NA>,16,95,<NA>,5.61,24.67,0.0,0.5,0.00,0.00,1.0,26.17,<NA>,<NA>,NaN,0.00
46911,2,2025-11-30 23:26:00,2025-11-30 23:42:00,NaN,<NA>,65,87,<NA>,2.91,14.46,0.0,0.5,3.89,0.00,1.0,23.35,<NA>,<NA>,NaN,0.75


## Ingest Data into postgres

In [5]:
from sqlalchemy import create_engine

POSTGRES_HOST = "localhost"
POSTGRES_PORT = 5433
POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "postgres"
POSTGRES_DB = "ny_taxi"
POSTGRES_TABLE = "green_taxi_data"

engine = create_engine(
    f"postgresql://{POSTGRES_USER}:{POSTGRES_PASSWORD}"
    f"@{POSTGRES_HOST}:{POSTGRES_PORT}/{POSTGRES_DB}"
)

In [6]:
df.head(n=0).to_sql(
    name=POSTGRES_TABLE,
    con=engine,
    if_exists="replace",
)
print("Table created\n")
print("Table scheme:")
print(pd.io.sql.get_schema(df, name=POSTGRES_TABLE, con=engine))

Table created

Table scheme:

CREATE TABLE green_taxi_data (
	"VendorID" BIGINT, 
	lpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	lpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	store_and_fwd_flag TEXT, 
	"RatecodeID" BIGINT, 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	passenger_count BIGINT, 
	trip_distance FLOAT(53), 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	payment_type BIGINT, 
	trip_type BIGINT, 
	congestion_surcharge FLOAT(53), 
	cbd_congestion_fee FLOAT(53)
)




In [7]:
df.to_sql(
    name=POSTGRES_TABLE,
    con=engine,
    if_exists="append",
)
print("Table completed\n")

Table completed

